In [1]:
%cd drive/MyDrive/Complexity-Measure/

/content/drive/MyDrive/Complexity-Measure


In [2]:
!pip install gower;

  Created wheel for gower: filename=gower-0.0.5-py3-none-any.whl size=4231 sha256=ffd09c6577b6af301cd0a4031c25290d6c575d1dc52f6613f797ccefc76f04b4
  Stored in directory: /root/.cache/pip/wheels/3e/f9/9a/67122a959a424e9cbb4557a8366c871a30e31cd75f0d003db4
Successfully built gower


In [3]:
from tensorflow.keras.applications import VGG16, Xception, ResNet50V2
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
import numpy as np
import cv2
from tensorflow.keras.datasets import mnist, cifar10
from measures import *
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA

In [4]:
from keras import backend as K
K.clear_session()

In [5]:
(x_train_mnist, y_train_mnist), (x_test, y_test) = mnist.load_data()
x_train_mnist.shape, y_train_mnist.shape

11493376/11490434 [==============================] - 0s 0us/step


((60000, 28, 28), (60000,))

In [6]:
(x_train_cifar, y_train_cifar), (x_test, y_test) = cifar10.load_data()
x_train_cifar.shape, y_train_cifar.shape

170500096/170498071 [==============================] - 2s 0us/step


((50000, 32, 32, 3), (50000, 1))

In [ ]:
def one_to_three_dim(x_train, dim):
  img_lst = []
  for i in range(len(x_train)):
    img = cv2.resize(x_train[i], dim, interpolation = cv2.INTER_AREA) 
    img = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    img_lst.append(img)
  rgb_arr = np.stack([img_lst],axis=4)
  rgb_arr_to_3d = np.squeeze(rgb_arr, axis=4)
  return rgb_arr_to_3d

In [ ]:
def feature_extractor(images, model, x, y, z):
  feature_list = []
  for i in range(len(images)):
    img_array = np.expand_dims(images[i], axis=0)
    features = model.predict(img_array)
    feature_list.append(features)
  feat_lt = np.reshape(feature_list, (-1, x*y*z))
  return feat_lt

In [ ]:
dim = (32, 32)
rgb_list = one_to_three_dim(x_train_mnist, dim)
print(rgb_list.shape)

(60000, 32, 32, 3)


In [ ]:
vgg = VGG16(weights='imagenet',
            include_top=False,
            input_shape=(32, 32, 3))

for layers in vgg.layers:
  layers.trainable=False

print(vgg.output)


58892288/58889256 [==============================] - 1s 0us/step
KerasTensor(type_spec=TensorSpec(shape=(None, 1, 1, 512), dtype=tf.float32, name=None), name='block5_pool/MaxPool:0', description="created by layer 'block5_pool'")


In [ ]:
vgg_features_mnist = feature_extractor(rgb_list[0:10000], vgg, 1, 1, 512)
np.save("vgg16_features_mnist.npy", vgg_features_mnist)
vgg_features_cifar = feature_extractor(x_train_cifar[0:10000], vgg, 1, 1, 512)
np.save("vgg16_features_cifar.npy", vgg_features_cifar)

In [16]:
PCA_X_mnist = x_train_mnist[0:10000]
PCA_Y_mnist = y_train_mnist[0:10000]
PCA_X_mnist = PCA_X_mnist/255
X_mnist_flat = PCA_X_mnist.reshape(-1, 28*28)
pca_mnist = PCA(0.9)
principalComponents_mnist = pca_mnist.fit_transform(X_mnist_flat)
principalComponents_mnist.shape

(10000, 85)

In [ ]:
compute_all_measures(principalComponents_mnist, PCA_Y_mnist)

F1 score:  0.3291913251953884
F1v score:  0.009387770114908826
F2 score:  1.7578469154652546e-07
F3 score:  3.7027312477213457








F4 score:  0.15668849313386993
R1 score:  0.0036440538195428363
R2 score:  0.0272
R3 score:  0.0129
N1 score:  0.1412
N2 score:  0.4292963703427572
N3 score:  0.07110000000000005
N4 score:  0.015000000000000013
LSC score:  0.99540873
Density score:  0.9233215921592159
ClsCoef score:  0.9202931282054626
Hubs score:  0.9941303588420946
T2 score:  0.0085
T3 score:  1.0493827160493827
T4 score:  117.6470588235294
C1 score:  0.0009846166111157073
C2 score:  0.0005563602001831258


In [17]:
PCA_X_cifar = x_train_cifar[0:10000]
PCA_Y_cifar = y_train_cifar[0:10000].reshape(10000)
PCA_X_cifar = PCA_X_cifar/255
X_cifar_flat = PCA_X_cifar.reshape(-1, 32*32*3)
pca_cifar = PCA(0.9)
principalComponents_cifar = pca_cifar.fit_transform(X_cifar_flat)
principalComponents_cifar.shape

(10000, 96)

In [ ]:
compute_all_measures(principalComponents_cifar, PCA_Y_cifar)

F1 score:  0.8593337119300521
F1v score:  0.04442644318520848
F2 score:  3.00057906810976e-08
F3 score:  4.902795564973681
F4 score:  0.6822646324459863
R1 score:  0.17487009631787268
R2 score:  0.5517
R3 score:  0.4747
N1 score:  0.7992
N2 score:  0.507231135804327
N3 score:  0.7303999999999999
N4 score:  0.23909999999999998
LSC score:  0.99992041
Density score:  0.9047292929292929
ClsCoef score:  0.904920420662385
Hubs score:  0.9905723421217026
T2 score:  0.0096
T3 score:  1.0434782608695652
T4 score:  104.16666666666667
C1 score:  0.00017168016136659414
C2 score:  9.649163956748641e-05


In [11]:
FTR_X_mnist = np.load("vgg16_features_mnist.npy")
FTR_X_mnist = FTR_X_mnist[0:3000]
Y_mnist = y_train_mnist[0:3000].reshape(3000)

In [12]:
compute_all_measures(FTR_X_mnist, Y_mnist)

F1 score:  0.2711741534621569


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/content/drive/MyDrive/Complexity-Measure/measures.py:159: RuntimeWarning: invalid value encountered in true_divide
  ratio = overlap_/range_


F1v score:  nan
F2 score:  0.0
F3 score:  0.0








F4 score:  0.0
R1 score:  0.0
R2 score:  0.0
R3 score:  0.0
N1 score:  0.21133333333333335
N2 score:  0.4259964227012036
N3 score:  0.1166666666666667
N4 score:  0.07566666666666666
LSC score:  0.9936381111111111
Density score:  0.899673224408136
ClsCoef score:  0.900232645730753
Hubs score:  0.983892474439038
T2 score:  0.17066666666666666
T3 score:  2.4150943396226414
T4 score:  5.859375
C1 score:  0.0013590429648728808
C2 score:  0.0007740423882753866


In [13]:
FTR_X_cifar = np.load("vgg16_features_cifar.npy")
FTR_X_cifar = FTR_X_cifar[0:3000]
Y_cifar = y_train_cifar[0:3000].reshape(3000)

In [14]:
compute_all_measures(FTR_X_cifar, Y_cifar)

F1 score:  0.7404271001611011


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/content/drive/MyDrive/Complexity-Measure/measures.py:159: RuntimeWarning: invalid value encountered in true_divide
  ratio = overlap_/range_


F1v score:  nan
F2 score:  0.0
F3 score:  0.0










F4 score:  0.0
R1 score:  0.0
R2 score:  0.0
R3 score:  0.0
N1 score:  0.721
N2 score:  0.5049166394624254
N3 score:  0.608
N4 score:  0.1266666666666667
LSC score:  0.9996926666666667
Density score:  0.9001158163832389
ClsCoef score:  0.898845411965076
Hubs score:  0.98248540919774
T2 score:  0.17066666666666666
T3 score:  1.3950953678474114
T4 score:  5.859375
C1 score:  0.0004004502936129084
C2 score:  0.0002273608255608517


In [18]:
compute_all_measures(principalComponents_mnist[0:3000], PCA_Y_mnist[0:3000])

F1 score:  0.33171914840455174
F1v score:  0.005406322899842282
F2 score:  1.697273214524083e-08
F3 score:  3.239426738146546






F4 score:  0.0
R1 score:  0.0006476492032878354
R2 score:  0.0036666666666666666
R3 score:  0.0016666666666666668
N1 score:  0.22266666666666668
N2 score:  0.43930449339441946
N3 score:  0.1136666666666667
N4 score:  0.012666666666666715
LSC score:  0.9937476666666667
Density score:  0.9447480271201512
ClsCoef score:  0.9371864477717142
Hubs score:  0.9925139826088891
T2 score:  0.028333333333333332
T3 score:  1.0625
T4 score:  35.294117647058826
C1 score:  0.0013590429648728808
C2 score:  0.0007740423882753866


In [19]:
compute_all_measures(principalComponents_cifar[0:3000], PCA_Y_cifar[0:3000])

F1 score:  0.8357211560360295
F1v score:  0.02219045827966898
F2 score:  1.5342934172728558e-09
F3 score:  4.728473643913595












F4 score:  0.0
R1 score:  0.09366829702308506
R2 score:  0.45366666666666666
R3 score:  0.38866666666666666
N1 score:  0.8286666666666667
N2 score:  0.5107003368140132
N3 score:  0.762
N4 score:  0.18000000000000005
LSC score:  0.9997692222222222
Density score:  0.914973657885962
ClsCoef score:  0.9167401681457262
Hubs score:  0.9852263046821048
T2 score:  0.032
T3 score:  1.0666666666666667
T4 score:  31.25
C1 score:  0.0004004502936129084
C2 score:  0.0002273608255608517
